<a href="https://colab.research.google.com/github/applejxd/colaboratory/blob/master/algorithm/NonLinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 例題

3次方程式
\begin{equation}
x^3+6x^2+21x+32=0
\end{equation}
の解は
\begin{equation}
x=-9^{1/3}+3^{1/3}-2,\ -9^{1/3}e^{i2\pi/3}+3^{1/3}e^{i4\pi/3}-2,\ -9^{1/3}e^{i4\pi/3}+3^{1/3}e^{i2\pi/3}-2
\end{equation}
または
\begin{equation}
x\sim-2.64,\ -1.68-3.05i,\ -1.68+3.05i
\end{equation}

In [8]:
def non_linear_func(x: float) -> float:
    return x**3 + 6*x**2 + 21*x + 32

x_real = -2.637834253

# 2分法

1. 解が含まれる領域 $[a,b]$ を事前に指定する
2. 中点を利用して領域を $[a,(a+b)/2]$ と $[(a+b)/2,b]$ の2つに分割する
3. 解を含む領域を中間値の定理で判定する
4. 解を含む領域を選択して新たに計算する領域として繰り返す
5. 領域幅が一定以下になったら終了

中間値の定理から領域 $[a,b]$ に対して $f(a)f(b)<0$ なら、この領域 $[a,b]$ に $f(x)=0$ の解が含まれる事がわかる。

In [11]:
from typing import Callable

def bisection(func: Callable[[float],float],
              left: float, right: float) -> float:
    '''
    二分法のアルゴリズム
    '''
    # 二分法で解けない場合
    if func(left) * func(right) >= 0:
        raise Exception('Cannot be solved by bisection method!')

    # 代数式の数値精度
    delta: float = 1e-10
    # 解の更新の幅
    epsilon: float = 5e-6

    middle: float = 0.
    while True:
        # 中点計算
        middle = (left + right)/2.

        # 中間値の定理から解を含む区間を選択
        if func(left) * func(middle) < 0:
            right = middle
        else:
            left = middle

        # 終了判定
        if (abs(func(middle)) < delta):
            print(f"finished: f(x)={abs(func(middle))}")
            break
        if (abs(left - right) < epsilon):
            print(f"finished: |left - right|={abs(left - right)} ")
            break
    
    middle = (left+right)/2.
    return middle

In [12]:
x = bisection(non_linear_func, -3.0, 0)
print(f"Δx = {x-x_real}, f(x)={non_linear_func(x)}")

finished: |left - right|=2.86102294921875e-06 
Δx = 1.1345612791480164e-06, f(x)=1.1593166991730186e-05
